# **Does estimation window size affects accuracy?**

In [1]:
#from google.colab import files
#uploaded = files.upload()

In [2]:
#!pip install category_encoders==2.*
#!pip install pandas-profiling==2.*

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
dataframe =  pd.read_csv('Melbourne_housing_FULL.csv')

In [5]:
dataframe.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0


In [6]:
# Our target is price. I personally think Features that related to price are ['Rooms','Bathroom','Landsize','Yearbuilt','Lattitude','Longtitude','Bedroom2'] and Other feautres are NOT related

# SO, only ['Price','Rooms','Bathroom','Landsize','Yearbuilt','Lattitude','Longtitude','Bedroom2','Date'] will be used.

In [7]:
dataframe=dataframe[['Price','Rooms','Bathroom','Landsize','YearBuilt','Lattitude','Longtitude','Bedroom2','Date']]

In [8]:
dataframe.dtypes

Price         float64
Rooms           int64
Bathroom      float64
Landsize      float64
YearBuilt     float64
Lattitude     float64
Longtitude    float64
Bedroom2      float64
Date           object
dtype: object

In [9]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Price       27247 non-null  float64
 1   Rooms       34857 non-null  int64  
 2   Bathroom    26631 non-null  float64
 3   Landsize    23047 non-null  float64
 4   YearBuilt   15551 non-null  float64
 5   Lattitude   26881 non-null  float64
 6   Longtitude  26881 non-null  float64
 7   Bedroom2    26640 non-null  float64
 8   Date        34857 non-null  object 
dtypes: float64(7), int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
dataframe.isna().sum() 

Price          7610
Rooms             0
Bathroom       8226
Landsize      11810
YearBuilt     19306
Lattitude      7976
Longtitude     7976
Bedroom2       8217
Date              0
dtype: int64

In [11]:
dataframe=dataframe.dropna(subset=['Price','Rooms','Bathroom','Landsize','YearBuilt','Lattitude','Longtitude','Bedroom2','Date'])

In [12]:
dataframe.isna().sum()

Price         0
Rooms         0
Bathroom      0
Landsize      0
YearBuilt     0
Lattitude     0
Longtitude    0
Bedroom2      0
Date          0
dtype: int64

In [13]:
len(dataframe[dataframe.duplicated()==True])

1

In [14]:
dataframe=dataframe.drop_duplicates()

In [15]:
len(dataframe[dataframe.duplicated()==True])

0

In [16]:
dataframe["Date"] = pd.to_datetime(dataframe["Date"],dayfirst=True)
dataframe['Date'].sort_values()

2       2016-02-04
6526    2016-02-04
6527    2016-02-04
6781    2016-02-04
10256   2016-02-04
           ...    
33523   2018-03-17
33525   2018-03-17
33529   2018-03-17
33513   2018-03-17
33366   2018-03-17
Name: Date, Length: 10612, dtype: datetime64[ns]

In [17]:
# The date the data has is from 16-02-04 to 18-03-17

In [18]:
# split
# train data has 2016,2017 data
# test data has 2018 data

In [19]:
test=dataframe[dataframe['Date']>'2017-12-31']
train = dataframe.drop(test.index)
display(dataframe.shape,train.shape,test.shape)

(10612, 9)

(9389, 9)

(1223, 9)

In [20]:
train=train.drop(axis=1,columns='Date')
test=test.drop(axis=1,columns='Date')

In [21]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=2)

In [22]:
dataframe.shape, train.shape , val.shape, test.shape

((10612, 9), (7511, 8), (1878, 8), (1223, 8))

In [23]:
target = 'Price'
features = train.drop(columns=[target]).columns
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [24]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(),
    StandardScaler(), 
    LinearRegression()
)

pipe.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=[], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[], return_df=True, verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [25]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
y_pred_val = pipe.predict(X_val)
print('Val accuracy: ', pipe.score(X_test, y_test))
print('MAE: ', mean_absolute_error(y_val, y_pred_val))
print('MSE: ', mean_squared_error(y_val, y_pred_val))

Val accuracy:  0.43827551205935267
MAE:  316224.78254723776
MSE:  243529639274.55112


In [26]:
y_pred_test = pipe.predict(X_test)
print('Test accuracy: ', pipe.score(X_test, y_test))
print('MAE: ', mean_absolute_error(y_test, y_pred_test))
print('MSE: ', mean_squared_error(y_test, y_pred_test))

Test accuracy:  0.43827551205935267
MAE:  344068.2427500459
MSE:  248152571338.3938
